In [13]:
import torch
import torch.nn.functional as F
import matplotlib as plt
%matplotlib inline

In [14]:
words = open('names.txt', 'r').read().splitlines()

In [15]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [16]:
def build_dataset(words):
    block_size = 3
    X, Y = [], []

    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix] # crop and append

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))
Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [17]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((30, 300), generator=g) # 300 is number of neurons - hyperparameter
b1 = torch.randn(300, generator=g)
W2 = torch.randn((300, 27), generator=g) # 300 is number of neurons - hyperparameter
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [18]:
sum(p.nelement() for p in parameters)

17697

In [19]:
for p in parameters:
    p.requires_grad = True

In [20]:
for i in range(50000):
    # Construct minibatch
    ix = torch.randint(0, Xtr.shape[0], (32,))
    
    # forward pass
    emb = C[Xtr[ix]]
    # torch.cat(torch.unbind(emb, 1), 1) Less efficient but same as below
    h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    lr = 0.01
    # update
    for p in parameters:
        p.data += -lr * p.grad
loss

tensor(2.0331, grad_fn=<NllLossBackward0>)

In [21]:
# forward pass
emb = C[Xtr]
# torch.cat(torch.unbind(emb, 1), 1) Less efficient but same as below
h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
loss

tensor(2.4309, grad_fn=<NllLossBackward0>)

In [22]:
# forward pass
emb = C[Xdev]
# torch.cat(torch.unbind(emb, 1), 1) Less efficient but same as below
h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss

tensor(2.4641, grad_fn=<NllLossBackward0>)

In [25]:
# training split, dev/validation split, test split
# 80%, 10%, 10%
block_size = 3
# Sampling
for _ in range(10):
    out = []
    context = [0]*block_size
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
    print(''.join(itos[i] for i in out))

swaelai.
noen.
elliuahischae.
auge.
ciiva.
ryamelenne.
jale.
duah.
brisa.
jay.
